In [2]:
import numpy as np
# help(np.linalg.norm)

In [3]:
def cal_Cosine_Similarity(x,y):
    print(x,y)
    # x = np.linalg.norm(x,ord=2,)
    return np.dot(x,y.T)/(np.linalg.norm(x,ord=2,)*np.linalg.norm(y,ord=2,))

x = np.arange(0,10,2)
y = np.arange(1,11,2)
out = cal_Cosine_Similarity(x,y)   
print(out)

[0 2 4 6 8] [1 3 5 7 9]
0.9949366763261819


# BN的计算过程：
## 训练阶段
![](./bn_train.jpg)

## 测试阶段
![](./bn_test.jpg)

**本案例说明：**

训练阶段：
1.假设输入的数据的格式是4行3列，4行数据分别输入到4个神经元。每个神经元处理3个batch的数据。
2.假定测试集和训练集具有同分布（均值、方差），在训练阶段需要计算训练集的均值和方差(采用滑动平均)。
3.在全局训练数据中看每个batch，不同的batch可能分布偏差很大，这样网络不好拟合数据不太稳定(covariate shift)，
需要捕捉到数据分布偏差的变化量，所以引入了伸缩因子γ和偏移因子β，这两个参数需要网络进行学习。

测试阶段：
输入的是一条条的数据

refer:
[Batch Normalization导读](https://blog.csdn.net/malefactor/article/details/51476961)

In [6]:


def cal_bn( data,static_args,flag,eps,gamma,beta):

    # uB_momentum = static_args["uB_momentum"]
    # varB_momentum = static_args["varB_momentum"]
    if flag == "train":
        print(data)
        uB = np.mean(data, axis=1,keepdims=True)
        print(uB)
        varB = np.var(data, axis=1,keepdims=True)
        static_args["uB_momentum"] = (1-momentum)*static_args["uB_momentum"] + momentum*uB
        static_args["varB_momentum"] = (1-momentum)*static_args["varB_momentum"] + momentum*varB
        data_ = (data - uB)/(np.sqrt(varB+eps))
        y_ = gamma* data_ + beta
        return y_
    else:
        y_ = (gamma * data)/(np.sqrt(static_args["varB_momentum"]+eps))-(beta-gamma*static_args["uB_momentum"]/(np.sqrt(static_args["varB_momentum"]+eps)))
        return y_


x = np.arange(0,12,1)
x.shape = (4, 3) # 设batch为4
print(x)
eps =  0.0001   # 常量
# 在测试时，输入的是单个数据，又假定测试集和训练集具有同分布（均值、方差）
momentum = 0.1
uB_momentum = np.array([1.1,[2.2],[3.3],[2.2]])
varB_momentum = np.array([[0.5],[0.75],[1.0],[1.5]])

# 伸缩因子用于减少训练和测试之间分布差异
gamma = np.array([[1.],[2.],[3.],[2.]])    # 需要学习的参数
beta = np.array([[1.5],[2.5],[3.5],[2.5]]) # 需要学习的参数

flag = "train" # 测试还是训练的标志位

static_args = {"uB_momentum":uB_momentum,"varB_momentum":varB_momentum}  # 需要在bn计算均值和方差时更新的参数
result = cal_bn(x,static_args,flag,eps,gamma,beta)
print(result)


[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
[[ 1.]
 [ 4.]
 [ 7.]
 [10.]]
[[ 0.27534697  1.5         2.72465303]
 [ 0.05069395  2.5         4.94930605]
 [-0.17395908  3.5         7.17395908]
 [ 0.05069395  2.5         4.94930605]]


In [7]:
x = np.array([[0.4805, 0.5710, 0.5412, 0.5067, 0.5402, 0.5237, 0.5242, 0.3718, 0.5531,0.5265, 0.4622, 0.6355, 0.4980, 0.4828, 0.5235, 0.6195, 0.6426, 0.5027,0.6080, 0.5534]])
x

y =  np.array([[ 0.5652, -0.2909, -0.3671, -1.1196, -0.4032, -0.0162, -1.1961, -0.7137,
                 0.4346,  0.8870,  0.1886, -0.7695,  0.2394,  0.3978, -0.5108,  0.3019,
                 -0.2587,  0.8602,  0.5676, -0.8220]])
y


array([[ 0.5652, -0.2909, -0.3671, -1.1196, -0.4032, -0.0162, -1.1961,
        -0.7137,  0.4346,  0.887 ,  0.1886, -0.7695,  0.2394,  0.3978,
        -0.5108,  0.3019, -0.2587,  0.8602,  0.5676, -0.822 ]])

In [8]:
x = np.array([[0.4805, 0.5710, 0.5412, 0.5067, 0.5402, 0.5237, 0.5242, 0.3718, 0.5531,0.5265, 0.4622, 0.6355, 0.4980, 0.4828, 0.5235, 0.6195, 0.6426, 0.5027,0.6080, 0.5534]])
eps =  0.0001   # 常量
varB_momentum = np.array([0.0020, 0.0025, 0.0025, 0.0037, 0.0032, 0.0027, 0.0025, 0.0044, 0.0029,
                          0.0015, 0.0031, 0.0023, 0.0029, 0.0028, 0.0028, 0.0020, 0.0024, 0.0020,
                          0.0020, 0.0024])
uB_momentum = np.array([0.5045, 0.5043, 0.5330, 0.4767, 0.5603, 0.4930, 0.4772, 0.5801, 0.5638,
                        0.4774, 0.4920, 0.5077, 0.5006, 0.5115, 0.5408, 0.4868, 0.5049, 0.5028,
                        0.4986, 0.5072])
gamma = np.array([1.1368, 0.8739, 1.0348, 0.8667, 1.1027, 1.0873, 1.1520, 0.8965, 0.9498,
                  1.0679, 1.0402, 1.1067, 0.9927, 0.7924, 1.0557, 1.0873, 0.9512, 1.1450,
                  1.0354, 1.1416]) # 需要学习的参数
beta = np.array([-0.7842, -0.5696, -0.6694, -0.5594, -0.7187,  0.6134,  0.7657, -0.5586,
                 -0.5946,  0.6025,  0.6065, -0.6100, -0.6112,  0.7322, -0.7373, -0.6379,
                 -0.6045,  0.7171,  0.7476, -0.6267]) # 需要学习的参数
# gamma * x
# np.sqrt(varB_momentum+eps)
y_ = (gamma * x)/(np.sqrt(varB_momentum+eps))+(beta-(gamma*uB_momentum/(np.sqrt(varB_momentum+eps))))
print(y_)
y__ = gamma*(x -uB_momentum)/(np.sqrt(varB_momentum+eps))+beta
print(y__)

[[-1.37956823  0.57354389 -0.5029884  -0.13760808 -1.10453042  1.24422478
   1.8275512  -3.34236972 -0.78014759  1.91334725  0.05852831  2.27705557
  -0.65832276  0.30989389 -1.07644672  2.5106505   2.0151048   0.71460141
   3.21941427  0.4281384 ]]
[[-1.37956823  0.57354389 -0.5029884  -0.13760808 -1.10453042  1.24422478
   1.8275512  -3.34236972 -0.78014759  1.91334725  0.05852831  2.27705557
  -0.65832276  0.30989389 -1.07644672  2.5106505   2.0151048   0.71460141
   3.21941427  0.4281384 ]]
